In [1]:
#1976-1979 NOTEBOOK

In [12]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from   sklearn.feature_extraction.text import TfidfVectorizer
from   sklearn.feature_selection import SelectKBest, mutual_info_classif
from   sklearn.linear_model import LogisticRegression, LinearRegression
from   sklearn.model_selection import cross_val_score
from   sklearn.preprocessing import StandardScaler

In [42]:
def metadata(date=''):
    '''
    a helper function for song_data.
    
    Creates metadata for the The Billboard Hot 100 chart for a specified week.
    
    date: a string, in the form "YYYY-MM-DD". For example, "2022-05-16" represents May 16, 2022. If no date specified, function
          will select the present chart
    returns: a pandas dataframe containing metadata for Billboard Hot 100 songs of the week of the specified date.
    columns: rank: rank of the week (1-100)
             date: a pandas datetime object. date of the chart as stated on the Billboard website, 
             which uses the Saturday to identify the week (so it is the same week as the user input, but the Saturday
             of that week),
             title: title of the song,
             artist1: main artist,
             artist2: a list of the rest of the artists. np.nan if there are none.
             peak_pos: peak position of the song,
             wks_chart: # of weeks the song has been on the chart
             b_url: url to the billboard chart
    '''
    lsongs=[]
    lartists=[]
    artist1=[]
    artist2=[]
    lpeak_pos=[]
    lwks_chart=[]
    
    URL='https://www.billboard.com/charts/hot-100/'+date

    page=requests.get(URL)
    soup=BeautifulSoup(page.content, 'lxml')
  
    ### get the first song, bc it's in a different div container
    song1 = soup.find("h3",id='title-of-a-story', class_="c-title a-no-trucate a-font-primary-bold-s u-letter-spacing-0021 u-font-size-23@tablet lrv-u-font-size-16 u-line-height-125 u-line-height-normal@mobile-max a-truncate-ellipsis u-max-width-245 u-max-width-230@tablet-only u-letter-spacing-0028@tablet")
    lsongs.append(song1.text.strip())
    
    ### get the first artist, bc it's in a different div container
    artistf=soup.find("span", class_="c-label a-no-trucate a-font-primary-s lrv-u-font-size-14@mobile-max u-line-height-normal@mobile-max u-letter-spacing-0021 lrv-u-display-block a-truncate-ellipsis-2line u-max-width-330 u-max-width-230@tablet-only u-font-size-20@tablet")
    lartists.append(artistf.text.strip())
    
    ### get the first peak position, bc it's in a different div container
    nums=soup.find_all('span', class_="c-label a-font-primary-bold-l a-font-primary-m@mobile-max u-font-weight-normal@mobile-max lrv-u-padding-tb-050@mobile-max u-font-size-32@tablet")
    nums1=[]
    for x in nums:
        nums1.append(x.text.strip())
        
    lpeak_pos.append(nums1[1])
    ### get the first weeks on chart, bc it's in a different div container
    lwks_chart.append(nums1[2])
    
    ### get last 99 songs
    songs = soup.find_all("h3", class_="c-title a-no-trucate a-font-primary-bold-s u-letter-spacing-0021 lrv-u-font-size-18@tablet lrv-u-font-size-16 u-line-height-125 u-line-height-normal@mobile-max a-truncate-ellipsis u-max-width-330 u-max-width-230@tablet-only", id="title-of-a-story")
    for song in songs:
        lsongs.append(song.text.strip())
    
    ### get last 99 artists
    artists = soup.find_all("span", class_="c-label a-no-trucate a-font-primary-s lrv-u-font-size-14@mobile-max u-line-height-normal@mobile-max u-letter-spacing-0021 lrv-u-display-block a-truncate-ellipsis-2line u-max-width-330 u-max-width-230@tablet-only")
    for artist in artists:
        lartists.append(artist.text.strip())
        
    ### get last 99 peak position
    all_num=[]
    peak_pos = soup.find_all("span", class_="c-label a-font-primary-m lrv-u-padding-tb-050@mobile-max")
    for num in peak_pos:
        all_num.append(num.text.strip())
    
    x=1
    for peak in all_num:
        if x <= len(all_num)-5:
            lpeak_pos.append(all_num[x])
            x=x+6

    ### get last 99 weeks on chart
    y=2
    for wk in all_num:
        if y <= len(all_num)-4:
            lwks_chart.append(all_num[y])
            y=y+6            
    
    ### get date as listed on the chart, aka the Saturday of the week of the user input
    date=soup.find('h2', id="section-heading")
    cdate=pd.to_datetime(date.text.strip().replace("Week of ",''))
    
    
    ### separate artists into artist1 and artist2
    for a in lartists:
        if ("X &" not in a) and ("X Featuring" not in a) and ("X /" not in a):
            a=a.replace(" X ",",")
        a=a.replace("Featuring",",")
        a=a.replace("&",",")   
        a=a.replace(" / ",",")
        a=a.replace('Feat.',',')
        a=a.replace('Feat',',')
        a=a.replace('FEAT.',',')
        a=a.replace('FEAT',',')
        a=a.replace('feat.',',')
        a=a.replace('feat',',')
        a=a.replace('featuring',',')
        List=a.split(",")
        artists = [i.strip() for i in List]
        artist1.append(artists[0])
        if len(artists)==1:
            artist2.append(np.nan)
        else:
            artist2.append(artists[1:])
    
    metadata=pd.DataFrame()
    metadata['rank']=(range(1,len(lsongs)+1)) ### get rank position
    metadata['date']=cdate
    metadata['title']=lsongs
    metadata['artist1']=artist1
    metadata['artist2']=artist2
    metadata['peak_pos']=lpeak_pos
    metadata['wks_chart']=lwks_chart
    metadata['b_url']=URL
    
#    metadata=append_lyrics(metadata)
#    metadata.reset_index(inplace=True, drop=True)

    return metadata

In [43]:
def append_lyrics(metadata):
    '''
    a helper function for song_data(). 
    gets the song lyrics for a given song. appends the
    song lyrics for a song from Genius.com to a "lyrics" column.
    
    If the function cannot find the song on the Genius lyrics website, it will drop the entire observation from the dataset.
    
    metadata: a pandas dataframe, created from song_data(). at the least contains
              the title column and the artist1 column.
    returns: a pandas dataframe of the original dataframe with a lyrics column and URL to the
             Genius website the lyrics were taken from.
    '''
    #get unique songs
    
    all_lyrics=[]
    all_URL=[]
    title=metadata.title.values
    artist1=metadata.artist1.values
    for x in range(len(title)):
        t=title[x]
        a=artist1[x]
  
        t=re.sub(r'[^\w\s]', '', t)
        a=re.sub(r'[!$/]', '-', a)
        a=re.sub(r'["\\#%&;\()*\[\]+,.:;<=>?@^_`{|}~]', '', a) #[\\]
        URL= "https://www.genius.com/"+a.replace(' ','-')+'-'+t.replace(' ','-')+'-lyrics'
        URL=URL.replace(",",'')
        URL=URL.replace('--','-')
        all_URL.append(URL)
    
    metadata['g_url']=all_URL
    
    unique_URL=np.unique(all_URL)
    
    real_URL=[]
    for u in unique_URL:
        page=requests.get(u)
        soup=BeautifulSoup(page.content, 'lxml')
        if 'Oops! Page not found' not in soup.text.strip():
            lyrics=soup.find_all('div', class_='Lyrics__Container-sc-1ynbvzw-6 jYfhrf')
            Lyrics = [re.sub(r"\[.*?\]",'',i.text.strip()) for i in lyrics]
            LYRICS=" ".join(Lyrics)
            all_lyrics.append(LYRICS)
            real_URL.append(u)
            print(u)
        
        else: 
            #print(URL)
            metadata.drop(metadata.index[metadata['g_url'] == u], inplace=True)
    
    unique_lyrics=pd.DataFrame()
    unique_lyrics['lyrics']=all_lyrics
    unique_lyrics['g_url']=real_URL
    
    data=metadata.merge(unique_lyrics, on='g_url', how='left')
    
    return data

In [46]:
def song_data(start,end):
    '''
    Creates a dataframe with all Billboard Hot 100 metadata and song lyrics (from Genius.com) for a specified range of dates.
    start: start date, a string "YYYY-MM-DD"
    end: end date, a string "YYYY-MM-DD"
    returns: a pandas dataframe.
    see metadata() and append_lyrics() for further documentation.
    '''
    dates=pd.date_range(start,end,freq='W-SAT')
    Dates=[date.strftime('%Y-%m-%d') for date in dates]
    #Dates=valid_dates(Dates)
    metadata_dfs=[]
    for date in Dates:
        metadata_dfs.append(metadata(date))
        #print(date)
    Metadata=pd.concat(metadata_dfs)
    Metadata.reset_index(inplace=True, drop=True)
    
    data=append_lyrics(Metadata)
    
    return data

In [45]:
%%time
songs1976=song_data('1976-01-01','1976-12-31')

1976-01-03
1976-01-10
1976-01-17
1976-01-24
1976-01-31
1976-02-07
1976-02-14
1976-02-21
1976-02-28
1976-03-06
1976-03-13
1976-03-20
1976-03-27
1976-04-03
1976-04-10
1976-04-17
1976-04-24
1976-05-01
1976-05-08
1976-05-15
1976-05-22
1976-05-29
1976-06-05
1976-06-12
1976-06-19
1976-06-26
1976-07-03
1976-07-10
1976-07-17
1976-07-24
1976-07-31
1976-08-07
1976-08-14
1976-08-21
1976-08-28
1976-09-04
1976-09-11
1976-09-18
1976-09-25
1976-10-02
1976-10-09
1976-10-16
1976-10-23
1976-10-30
1976-11-06
1976-11-13
1976-11-20
1976-11-27
1976-12-04
1976-12-11
1976-12-18
1976-12-25
https://www.genius.com/10cc-Art-For-Arts-Sake-lyrics
https://www.genius.com/10cc-Im-Mandy-Fly-Me-lyrics
https://www.genius.com/ABBA-Dancing-Queen-lyrics
https://www.genius.com/ABBA-Fernando-lyrics
https://www.genius.com/ABBA-I-Do-I-Do-I-Do-I-Do-I-Do-lyrics
https://www.genius.com/ABBA-Mamma-Mia-lyrics
https://www.genius.com/Aerosmith-Dream-On-lyrics
https://www.genius.com/Aerosmith-Home-Tonight-lyrics
https://www.genius.com/A

In [48]:
songs_rest=song_data('1977-01-01','1979-12-31')

https://www.genius.com/10cc-Dreadlock-Holiday-lyrics
https://www.genius.com/10cc-For-You-And-I-lyrics
https://www.genius.com/10cc-Good-Morning-Judge-lyrics
https://www.genius.com/10cc-People-In-Love-lyrics
https://www.genius.com/10cc-The-Things-We-Do-For-Love-lyrics
https://www.genius.com/A-Taste-Of-Honey-Boogie-Oogie-Oogie-lyrics
https://www.genius.com/A-Taste-Of-Honey-Do-It-Good-lyrics
https://www.genius.com/ABBA-Angeleyes-lyrics
https://www.genius.com/ABBA-Chiquitita-lyrics
https://www.genius.com/ABBA-Dancing-Queen-lyrics
https://www.genius.com/ABBA-Does-Your-Mother-Know-lyrics
https://www.genius.com/ABBA-Knowing-Me-Knowing-You-lyrics
https://www.genius.com/ABBA-Money-Money-Money-lyrics
https://www.genius.com/ABBA-Take-A-Chance-On-Me-lyrics
https://www.genius.com/ABBA-The-Name-Of-The-Game-lyrics
https://www.genius.com/AC-DC-Highway-To-Hell-lyrics
https://www.genius.com/Ace-Frehley-New-York-Groove-lyrics
https://www.genius.com/Addrisi-Brothers-Does-She-Do-It-Like-She-Dances-lyrics
ht

In [51]:
both=pd.concat([songs1976, songs_rest])

In [52]:
both.reset_index(inplace=True, drop=True)

In [53]:
both.to_csv('songs1976_1979.csv')

In [47]:
page=requests.get('https://www.billboard.com/charts/hot-100/1976-12-04/')
soup=BeautifulSoup(page.content, 'lxml')
song1 = soup.find("h3",id='title-of-a-story', class_="c-title a-no-trucate a-font-primary-bold-s u-letter-spacing-0021 u-font-size-23@tablet lrv-u-font-size-16 u-line-height-125 u-line-height-normal@mobile-max a-truncate-ellipsis u-max-width-245 u-max-width-230@tablet-only u-letter-spacing-0028@tablet")
song1.text.strip()
songs = soup.find_all("h3", class_="c-title a-no-trucate a-font-primary-bold-s u-letter-spacing-0021 lrv-u-font-size-18@tablet lrv-u-font-size-16 u-line-height-125 u-line-height-normal@mobile-max a-truncate-ellipsis u-max-width-330 u-max-width-230@tablet-only", id="title-of-a-story")
lsongs=[]
for song in songs:
    lsongs.append(song.text.strip())

In [32]:
x=2
for l in lsongs:
    print(x,l)
    x=x+1

2 The Rubberband Man
3 Love So Right
4 Muskrat Love
5 The Wreck Of The Edmund Fitzgerald
6 You Don't Have To Be A Star (To Be In My Show)
7 Beth/Detroit Rock City
8 More Than A Feeling
9 Nadia's Theme (The Young And The Restless)
10 You Are The Woman
11 Nights Are Forever Without You
12 You Make Me Feel Like Dancing
13 Disco Duck (Part I)
14 Just To Be Close To You
15 Sorry Seems To Be The Hardest Word
16 I Never Cry
17 The Best Disco In Town
18 Stand Tall
19 Love Me
20 After The Lovin'
21 Love Ballad
22 Hot Line
23 Rock'n Me
24 Dazz
25 Do You Feel Like We Do
26 Livin' Thing
27 Hello Old Friend
28 It's A Long Way There
29 If You Leave Me Now
30 Fernando
31 (Don't Fear) The Reaper
32 Magic Man
33 I Only Want To Be With You
34 Jeans On
35 Shake Your Rump To The Funk
36 Lost Without Your Love
37 Whenever I'm Away From You
38 You Are My Starship
39 I Wish
40 Don't Take Away The Music
41 Car Wash
42 Saturday Nite
43 Enjoy Yourself
44 I Like Dreamin'
45 Do What You Want, Be What You Are
46 D

In [43]:
num=list(range(2000,2022))

In [55]:
data=[]
for year in num:
    file='songs'+str(year)+'.csv'
    df=pd.read_csv(file)
    print("year",year)
    print(len(df))
    data.append(df)

year 2000
4671
year 2001
5200
year 2002
5200
year 2003
5200
year 2004
5200
year 2005
4955
year 2006
4949
year 2007
4700
year 2008
4787
year 2009
4820
year 2010
4902
year 2011
4949
year 2012
5148
year 2013
5121
year 2014
5175
year 2015
5168
year 2016
4976
year 2017
5181
year 2018
4917
year 2019
5179
year 2020
5188
year 2021
4631


In [2]:
songs=pd.concat(data)
songs.reset_index(inplace=True,drop=True)

NameError: name 'pd' is not defined

In [15]:
def valid_dates(datelist):
    '''
    a preprocessing function to check if a chart exists for a given date.
    e.g. https://www.billboard.com/charts/hot-100/2011-08-06/ does not have a chart, but https://www.billboard.com/charts/hot-100/2011-08-13/ does.
    removes broken links from a list of dates.
    i.e. if a Billboard chart does not exist for a date specified in a list, remove the date from the list.
    
    datelist: a list of strings, each dates in YYYY-MM-DD format.
    returns: a list of valid dates from the datelist.
    '''
    valid=[]
    for date in datelist:
        URL='https://www.billboard.com/charts/hot-100/'+date
        page=requests.get(URL)
        soup=BeautifulSoup(page.content, 'lxml')
        song1 = soup.find("h3",id='title-of-a-story', class_="c-title a-no-trucate a-font-primary-bold-s u-letter-spacing-0021 u-font-size-23@tablet lrv-u-font-size-16 u-line-height-125 u-line-height-normal@mobile-max a-truncate-ellipsis u-max-width-245 u-max-width-230@tablet-only u-letter-spacing-0028@tablet")
        if song1 != None:
            valid.append(date)
    return valid

In [8]:
DATES=valid_dates(Dates)
print(len(Dates))
len(DATES)

52


52

In [72]:
songs=[]
for date in Dates:
    songs.append(song_data(date))
    print(date)

KeyboardInterrupt: 

In [10]:
df=pd.concat(songs)
df.reset_index(inplace=True,drop=True)
df.to_csv('songs1990.csv')

In [11]:
df

,rank,date,title,artist1,artist2,peak_pos,wks_chart,b_url,lyrics,g_url
0,1,2021-01-02,All I Want For Christmas Is You,Mariah Carey,NaN,1,43,https://www.billboard.com/charts/hot-100/2021-...,I don't want a lot for ChristmasThere is just ...,https://www.genius.com/Mariah-Carey-All-I-Want...
1,2,2021-01-02,Rockin' Around The Christmas Tree,Brenda Lee,NaN,2,38,https://www.billboard.com/charts/hot-100/2021-...,Rockin' around the Christmas treeAt the Christ...,https://www.genius.com/Brenda-Lee-Rockin-Aroun...
2,3,2021-01-02,Jingle Bell Rock,Bobby Helms,NaN,3,35,https://www.billboard.com/charts/hot-100/2021-...,"Jingle bell, jingle bell, jingle bell rockJing...",https://www.genius.com/Bobby-Helms-Jingle-Bell...
3,4,2021-01-02,A Holly Jolly Christmas,Burl Ives,NaN,4,19,https://www.billboard.com/charts/hot-100/2021-...,(Ding-dong-ding)(Ding-dong-ding)(Ding-dong-din...,https://www.genius.com/Burl-Ives-A-Holly-Jolly...
4,5,2021-01-02,It's The Most Wonderful Time Of The Year,Andy Williams,NaN,5,20,https://www.billboard.com/charts/hot-100/2021-...,It's the most wonderful time of the yearWith t...,https://www.genius.com/Andy-Williams-Its-The-M...
...,...,...,...,...,...,...,...,...,...,...
4626,95,2021-12-25,Freedom Was A Highway,Jimmie Allen,[Brad Paisley],76,10,https://www.billboard.com/charts/hot-100/2021-...,"(Oh-oh, oh-oh, woo)(Oh-oh, oh-oh)Sunset throug...",https://www.genius.com/Jimmie-Allen-Freedom-Wa...
4627,96,2021-12-25,No Love,Summer Walker,[SZA],13,6,https://www.billboard.com/charts/hot-100/2021-...,"Oh, ooh woahOh-oh, yeahYeah, yeah, yeahIf I ha...",https://www.genius.com/Summer-Walker-No-Love-l...
4628,97,2021-12-25,Bad Man (Smooth Criminal),Polo G,NaN,49,5,https://www.billboard.com/charts/hot-100/2021-...,"Lil Capalot, bitch, haSmooth criminal, Mike Ja...",https://www.genius.com/Polo-G-Bad-Man-Smooth-C...
4629,98,2021-12-25,Feel Alone,Juice WRLD,NaN,98,1,https://www.billboard.com/charts/hot-100/2021-...,"Smokin' this dope, relaxin'I ain't gon' lie, b...",https://www.genius.com/Juice-WRLD-Feel-Alone-l...
